# Web Scraping con Python y Selenium en Amazon

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time



driver = webdriver.Chrome()
driver.get("https://www.amazon.com/es")


time.sleep(5)

try:
    
    buscar_amazon = driver.find_element(By.ID, "twotabsearchtextbox")
    buscar_amazon.click()
    buscar_amazon.send_keys("book")
    buscar_amazon.submit()
    
except:
    pass

time.sleep(5)

try:
    
    buscar_libros = driver.find_element(By.PARTIAL_LINK_TEXT, "Libros en Español")
    buscar_libros.click()
    
except:
    pass

try:
    
    Libros_mas_vendidos = driver.find_element(By.PARTIAL_LINK_TEXT, "Ver más")
    Libros_mas_vendidos.click()
    
except:
    pass


# Encontrar los primeros 30 elementos que coinciden con el ID "gridItemRoot"
elementos = driver.find_elements(By.XPATH, '//div[@id="gridItemRoot"]')[:30]

# Listas para almacenar autores, títulos, comentarios y IDs
autores_por_libro = []
titulos = []
comentarios_autores = []
id_libros = []



# Recorrer los elementos para obtener los Ids
for elemento in elementos:
    libro_id_elemento = elemento.find_element(By.XPATH, './/*[@data-asin]')
    libro_id = libro_id_elemento.get_attribute("data-asin")
    id_libros.append(libro_id)

# Recorrer cada elemento y acceder al enlace del libro
for elemento in elementos:
    enlace = elemento.find_element(By.XPATH, './/a[@class="a-link-normal"]')
    url_libro = enlace.get_attribute('href')

    # Abrir el enlace del libro en una nueva pestaña
    driver.execute_script('window.open(arguments[0])', url_libro)

    # Cambiar el enfoque al nuevo libro abierto
    driver.switch_to.window(driver.window_handles[-1])

    # Extraer el título del libro
    elemento_titulo = driver.find_element(By.ID, 'productTitle')
    titulo = elemento_titulo.text.strip()
    titulos.append(titulo)
  

    # Extraer los autores del libro
    elementos_autores = driver.find_elements(By.XPATH, '//span[@class="author notFaded"]/a[@class="a-link-normal"]')
    nombres_autores = [elemento_autor.text.strip() for elemento_autor in elementos_autores]
    
    # Agregar la lista de autores al listado de autores por libro
    autores_por_libro.append(nombres_autores)
 

    # Extraer los comentarios del libro
    elementos_comentarios = driver.find_elements(By.XPATH, '//a[@data-hook="review-title"]/span')
    texto_comentarios = [elemento_comentario.text.strip() for elemento_comentario in elementos_comentarios]

    # Crear un diccionario con el autor y los comentarios
    diccionario_comentarios = {'autor': nombres_autores[0], 'comentarios': texto_comentarios}
    comentarios_autores.append(diccionario_comentarios)

    # Esperar 3 segundos entre cada libro
    time.sleep(3)

    # Cerrar la pestaña actual y volver a la página anterior
    driver.close()
    driver.switch_to.window(driver.window_handles[0])

# Cerrar el navegador
driver.quit()

##### Imprimir los autores, titulos y comentarios

In [2]:
print("Autores por libro:")
for autores in autores_por_libro:
    print(autores)

print("Títulos:")
for titulo in titulos:
    print(titulo)

print("Comentarios por autores:")
for comentarios in comentarios_autores:
    print(comentarios)

Autores por libro:
['James Clear', 'Arturo Guerrero']
['David Shannon']
['Priscy Escoto']
['Marián Rojas Estapé']
['Editors of Merriam-Webster', 'Merriam-Webster']
['Sophie Saint Rose', 'Sonia López Rodríguez']
['Ann Corcorane']
['Bill Martin, Jr.']
['Robert T. Kiyosaki']
['Ana Galán', 'Pablo Pino']
['B&H Español Editorial Staff']
['Robert T. Kiyosaki']
['Frank Suarez']
['Jane Werner Watson', 'Eloise Wilkin']
['Kelbin Torres']
['Melissa Stewart']
['Dr. Shinichi Suzuki']
['Rayo Guzmán', 'Adriana Galindo']
['Melissa Stewart']
['Reina Valera 1960']
['Dale Carnegie', 'Diego Guerrero']
['Eric Carle']
['Brian Tracy', 'Dave Ramos']
['Bill Martin, Jr.', 'Eric Carle']
['Mario Alonso Puig', 'Planeta Audio']
['Gabriel García Márquez']
['Brian Alba', 'Tobias Olveira']
['Reina Valera Revisada 1960']
['Helen Gregory']
['Marian Rojas Estapé']
Títulos:
Hábitos atómicos: Un método sencillo y comprobado para desarrollar buenos hábitos y eliminar los malos
David va a la escuela (David Goes to School) (Da

##### Verificar que tengan la misma longitud para crear un Dataframe

In [3]:
print(len(autores_por_libro))
print(len(titulos))
print(len(comentarios_autores))
print(len(id_libros))

30
30
30
30


##### Crear el Dataframe

In [5]:
import pandas as pd

data = {"ID": id_libros, 'Autor': autores_por_libro, 'Título': titulos, 'Comentarios': comentarios_autores}
df = pd.DataFrame(data)

df.head()

,ID,Autor,Título,Comentarios
0,B08JWXX27C,"[James Clear, Arturo Guerrero]",Hábitos atómicos: Un método sencillo y comprob...,"{'autor': 'James Clear', 'comentarios': ['', '..."
1,B075LW4GSQ,[David Shannon],David va a la escuela (David Goes to School) (...,"{'autor': 'David Shannon', 'comentarios': ['',..."
2,1961809532,[Priscy Escoto],Soy Abundancia: Guía para manifestar abundanci...,"{'autor': 'Priscy Escoto', 'comentarios': []}"
3,B088Y1YMWV,[Marián Rojas Estapé],Cómo hacer que te pasen cosas buenas: Entiende...,"{'autor': 'Marián Rojas Estapé', 'comentarios'..."
4,1596951656,"[Editors of Merriam-Webster, Merriam-Webster]",Merriam-Webster Webster’s Spanish-English Dict...,"{'autor': 'Editors of Merriam-Webster', 'comen..."


In [6]:
df.head(30)

,ID,Autor,Título,Comentarios
0,B08JWXX27C,"[James Clear, Arturo Guerrero]",Hábitos atómicos: Un método sencillo y comprob...,"{'autor': 'James Clear', 'comentarios': ['', '..."
1,B075LW4GSQ,[David Shannon],David va a la escuela (David Goes to School) (...,"{'autor': 'David Shannon', 'comentarios': ['',..."
2,1961809532,[Priscy Escoto],Soy Abundancia: Guía para manifestar abundanci...,"{'autor': 'Priscy Escoto', 'comentarios': []}"
3,B088Y1YMWV,[Marián Rojas Estapé],Cómo hacer que te pasen cosas buenas: Entiende...,"{'autor': 'Marián Rojas Estapé', 'comentarios'..."
4,1596951656,"[Editors of Merriam-Webster, Merriam-Webster]",Merriam-Webster Webster’s Spanish-English Dict...,"{'autor': 'Editors of Merriam-Webster', 'comen..."
5,B0CPD89QMQ,"[Sophie Saint Rose, Sonia López Rodríguez]",No soy lo que esperabas (Spanish Edition),"{'autor': 'Sophie Saint Rose', 'comentarios': ..."
6,B07DNGC252,[Ann Corcorane],Caliente y Frío (Wonder Readers Spanish Emerge...,"{'autor': 'Ann Corcorane', 'comentarios': ['',..."
7,B0191KMHFU,"[Bill Martin, Jr.]","Oso pardo, oso pardo, ¿qué ves ahí?: / Brown B...","{'autor': 'Bill Martin, Jr.', 'comentarios': [..."
8,B01H4E8PZQ,[Robert T. Kiyosaki],"Padre Rico, Padre Pobre [Rich Dad, Poor Dad]: ...","{'autor': 'Robert T. Kiyosaki', 'comentarios':..."
9,0545328632,"[Ana Galán, Pablo Pino]",Qué cosas dice mi abuela (The Things My Grandm...,"{'autor': 'Ana Galán', 'comentarios': ['', 'Be..."
